# MANUAL NET ARCHITECTURE

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import glob
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from PIL import Image

from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
Amoeba = 'DATASET/TRAIN/Amoeba'
Euglena = 'DATASET/TRAIN/Euglena'
Hydra = 'DATASET/TRAIN/Hydra'
Paramecium = 'DATASET/TRAIN/Paramecium'
Rod_bacteria = 'DATASET/TRAIN/Rod_bacteria'
Spherical_bacteria = 'DATASET/TRAIN/Spherical_bacteria'
Spiral_bacteria = 'DATASET/TRAIN/Spiral_bacteria'
Yeast = 'DATASET/TRAIN/Yeast'

In [ ]:
def plot_images(item_dir, n=6):
    all_item_dir = os.listdir(item_dir)  # Get all items in the directory
    item_files = [os.path.join(item_dir, file) for file in all_item_dir][:n]  # Get file paths for first n items
    
    plt.figure(figsize=(80, 40))  # Create a large figure for plotting images
    for idx, img_path in enumerate(item_files):
        plt.subplot(3, n, idx+1)  # Create subplots
        img = plt.imread(img_path)  # Read image using matplotlib
        plt.imshow(img, cmap='gray')  # Display image in grayscale
        plt.axis('off')  # Turn off axis
    
    plt.tight_layout()  # Adjust subplot parameters to give specified padding

def image_details_print(data,path):
    print('======== Images in: ', path)  # Print section header
    for key,values in data.items():
        print(key,':\t', values)  # Print key-value pairs for image details


def images_details(path):
    files=[f for f in glob.glob(path + "**/*.*", recursive=True)]  # Get list of all image files in directory and subdirectories
    data={}  # Initialize dictionary to store image details
    data['Images_count']=len(files)  # Count total number of images
    data['Min_width']=10**100  # Initialize minimum width with a very large number
    data['Max_width']=0  # Initialize maximum width with 0
    data['Min_height']=10**100  # Initialize minimum height with a very large number
    data['Max_height']=0  # Initialize maximum height with 0
    
    for f in files:
        img=Image.open(f)  # Open image using PIL
        width,height=img.size  # Get width and height of the image
        data['Min_width']=min(width,data['Min_width'])  # Update minimum width
        data['Max_width']=max(width, data['Max_width'])  # Update maximum width
        data['Min_height']=min(height, data['Min_height'])  # Update minimum height
        data['Max_height']=max(height, data['Max_height'])  # Update maximum height
        
    image_details_print(data,path)  # Print image details
    

In [ ]:
print("")
print("TRAINING DATA FOR Amoeba:")
print("")
images_details(Amoeba)
print("")
plot_images(Amoeba, 10) 

In [ ]:
print("")
print("TRAINING DATA FOR Euglena:")
print("")
images_details(Euglena)
print("")
plot_images(Euglena, 10)

In [ ]:
print("")
print("TRAINING DATA FOR Hydra:")
print("")
images_details(Hydra)
print("")
plot_images(Hydra, 10)

In [ ]:
print("")
print("TRAINING DATA FOR Yeast:")
print("")
images_details(Yeast)
print("")
plot_images(Yeast, 10)

In [ ]:
print("")
print("TRAINING DATA FOR Paramecium:")
print("")
images_details(Paramecium)
print("")
plot_images(Paramecium, 10)

In [ ]:
print("")
print("TRAINING DATA FOR Rod_bacteria:")
print("")
images_details(Rod_bacteria)
print("")
plot_images(Rod_bacteria, 10)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0,1] range
    shear_range=0.2,  # Apply shear transformation
    zoom_range=0.2,  # Apply zoom transformation
    horizontal_flip=True  # Apply horizontal flip
)

training_set = train_datagen.flow_from_directory(
    'dataset/train',  # Path to the training dataset directory
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,  # Number of samples per gradient update
    class_mode='categorical'  # Type of label array (categorical for one-hot encoded labels)
)


In [ ]:
test_set = test_datagen.flow_from_directory(
    'dataset/test',    # Path to the test dataset directory
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,            # Number of samples per batch
    class_mode='categorical'  # Type of label array (categorical for one-hot encoded labels)
)


In [ ]:


# Create a sequential model
Classifier = Sequential()

# Add a convolutional layer with 32 filters, each filter size is 3x3,
# input images are expected to be of size 224x224 pixels with 3 channels (RGB),
# and ReLU activation function is used
Classifier.add(Convolution2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))

# Add a max pooling layer with a pooling window size of 2x2
Classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the previous layer
Classifier.add(Flatten())

# Add a fully connected layer with 38 neurons and ReLU activation function
Classifier.add(Dense(38, activation='relu'))

# Add another fully connected layer with 8 neurons and softmax activation function
# Softmax is used for multi-class classification to output probabilities for each class
Classifier.add(Dense(8, activation='softmax'))

# Compile the model with RMSprop optimizer, categorical crossentropy loss function,
# and accuracy as the metric to track during training and testing
Classifier.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#model save 
model_path = "MANUAL.h5"

callbacks = [
    ModelCheckpoint(model_path, monitor='accuracy', verbose=1, save_best_only=True)
]

In [ ]:
epochs = 10
batch_size = 512

In [ ]:
#### Fitting the model
history = Classifier.fit(
           training_set, steps_per_epoch=training_set.samples // batch_size, 
           epochs=epochs, 
           validation_data=test_set,validation_steps=test_set.samples // batch_size,
           callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt

def graph():
    #Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

graph()

In [ ]:
import matplotlib.pyplot as plt

def graph():
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
graph()